<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

### Libraries

In [68]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
import gc

In [69]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

### Loading Data

#### Historical data

In [3]:
%%time
transactions1 = pd.read_csv("../data/historical_transactions.csv")  #,usecols=columns)
transactions1['purchase_date']=pd.to_datetime(transactions1['purchase_date'])
transactions1['month']=transactions1['purchase_date'].dt.month
transactions1['year']=transactions1['purchase_date'].dt.year
transactions1['weekofyear']=transactions1['purchase_date'].dt.weekofyear
transactions1['dayofweek']=transactions1['purchase_date'].dt.dayofweek


Wall time: 7min 29s


In [4]:
transactions1.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month,year,weekofyear,dayofweek,week
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,6,2017,25,6,25
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,7,2017,28,5,28
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,8,2017,32,2,32
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,9,2017,35,5,35
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,3,2017,10,4,10


In [10]:
auth={'Y':1,'N':0}
transactions1['authorized']=transactions1['authorized_flag'].map(auth)

In [37]:
transactions1['week']=transactions1['purchase_date'].dt.week
transactions1['dayofyear']=transactions1['purchase_date'].dt.dayofyear
transactions1['day']=transactions1['purchase_date'].dt.day

#### New Transactions data

In [5]:
%%time
transactions2 = pd.read_csv("../data/new_merchant_transactions.csv")  #,usecols=columns)
transactions2['purchase_date']=pd.to_datetime(transactions2['purchase_date'])
transactions2['month']=transactions2['purchase_date'].dt.month
transactions2['year']=transactions2['purchase_date'].dt.year
transactions2['weekofyear']=transactions2['purchase_date'].dt.weekofyear
transactions2['dayofweek']=transactions2['purchase_date'].dt.dayofweek

transactions2.head()

Wall time: 27.2 s


In [36]:
transactions2['week']=transactions2['purchase_date'].dt.week
transactions2['dayofyear']=transactions2['purchase_date'].dt.dayofyear
transactions2['day']=transactions2['purchase_date'].dt.day

In [6]:
transactions2.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month,year,weekofyear,dayofweek,week
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19,3,2018,10,6,10
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19,3,2018,12,0,12
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14,4,2018,17,3,17
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8,3,2018,10,2,10
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29,3,2018,12,3,12


In [11]:
transactions2['authorized']=transactions2['authorized_flag'].map(auth)

In [13]:
transactions2.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id', 'month', 'year', 'weekofyear', 'dayofweek', 'week',
       'authorized'],
      dtype='object')

### Feature Extraction

#### Card Level Features

##### History

In [38]:
clf={'authorized':'sum',
    'city_id':'nunique',
     'installments':'sum',
     'merchant_category_id':'nunique',
     'merchant_id':'nunique',
     'month_lag':['nunique', 'max',lambda x: max(x)-min(x)],
     'purchase_amount':['sum', 'mean','max','min',lambda x: max(x)-min(x),'count'],
     'state_id':'nunique',
     'subsector_id':'nunique',
     'year':'nunique',
     'weekofyear':'nunique',
     'month':'max',
     'dayofyear':'nunique'
    }

In [47]:
%%time
card_summary_history=transactions1.groupby('card_id').agg(clf)
card_summary_history.columns=[x[0]+"_"+x[1] for x in card_summary_history.columns]
card_summary_history=card_summary_history.reset_index()
card_summary_history.to_csv("../Fresh/card_summary_history.csv",index=False)

Wall time: 6min 42s


##### Future

In [48]:
%%time
card_summary_future=transactions2.groupby('card_id').agg(clf)
card_summary_future.columns=[x[0]+"_"+x[1] for x in card_summary_future.columns]
card_summary_future=card_summary_future.reset_index()
card_summary_future.to_csv("../Fresh/card_summary_future.csv",index=False)

Wall time: 1min 2s


In [46]:
card_summary_future.columns

Index(['card_id', 'authorized_sum', 'city_id_nunique', 'installments_sum',
       'merchant_category_id_nunique', 'merchant_id_nunique',
       'month_lag_nunique', 'month_lag_max', 'month_lag_<lambda>',
       'purchase_amount_sum', 'purchase_amount_mean', 'purchase_amount_max',
       'purchase_amount_min', 'purchase_amount_<lambda>',
       'purchase_amount_count', 'state_id_nunique', 'subsector_id_nunique',
       'year_nunique', 'weekofyear_nunique', 'month_max', 'dayofyear_nunique'],
      dtype='object')

#### Card and category1,2,3 features

In [50]:
%%time
transactions3=pd.concat([transactions1,transactions2],axis=0)

##### Category1

In [62]:
%%time
category1=transactions3.groupby(['card_id','category_1']).\
    agg({'merchant_category_id':'nunique','merchant_id':'nunique','purchase_amount':'sum','purchase_date':'count'}).reset_index()

for i in category1['category_1'].unique():
    category1_1=category1[category1['category_1']==i]
    category1_1.drop('category_1',inplace=True,axis=1)
    category1_1.columns=[x+i for x in category1_1.columns]
    category1_1=category1_1.rename(columns={'card_id'+i:'card_id'})
    category1_1.to_csv("../Fresh/category1_all"+i+".csv", index=False)

C:\Users\sai.allaka\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Wall time: 1min 59s


##### Category2

In [67]:
%%time
transactions3['category_2']=transactions3['category_2'].fillna(0)
category2=transactions3.groupby(['card_id','category_2']).\
    agg({'merchant_category_id':'nunique','merchant_id':'nunique','purchase_amount':'sum','month':'count'}).reset_index()

for i in category2['category_2'].unique():
    category2_2=category2[category2['category_2']==i]
    category2_2.drop('category_2',inplace=True,axis=1)
    category2_2.columns=[str(x)+str(i) for x in category2_2.columns]
    category2_2=category2_2.rename(columns={'card_id'+str(i):'card_id'})
    category2_2.to_csv("../Fresh/category2_all"+str(i)+".csv", index=False)

C:\Users\sai.allaka\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Wall time: 1min 56s


##### Category3

In [55]:
%%time
category3=transactions3.groupby(['card_id','category_3']).\
    agg({'merchant_category_id':'nunique','merchant_id':'nunique','purchase_amount':'sum','month':'count'}).reset_index()

for i in category3['category_3'].unique():
    category3_3=category3[category3['category_3']==i]
    category3_3.drop('category_3',inplace=True,axis=1)
    category3_3.columns=[x+i for x in category3_3.columns]
    category3_3=category3_3.rename(columns={'card_id'+i:'card_id'})
    category3_3.to_csv("../Fresh/category3_all"+i+".csv", index=False)

C:\Users\sai.allaka\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Wall time: 1min 56s


In [56]:
del([category3,category2,category1,category3_3,category2_2,category1_1])
gc.collect()

21

#### Card - Time level features

In [71]:
clf={'authorized':'sum',
     'city_id':'nunique',
     'installments':'sum',
     'merchant_category_id':'nunique',
     'merchant_id':'nunique',
     'purchase_amount':['sum', 'mean','max','min',lambda x: max(x)-min(x),'count'],
     'state_id':'nunique',
     'subsector_id':'nunique',
     'week':'nunique',
     'dayofyear':'nunique'
    }

In [82]:
%%time
card_summary_month_history=transactions1.groupby(['card_id','month_lag']).agg(clf)
card_summary_month_history.columns=[x[0]+"_"+x[1] for x in card_summary_month_history.columns]
card_summary_month_history=card_summary_month_history.reset_index()
stats=['mean','max','min',lambda x: max(x)-min(x)]
cols=[x for x in card_summary_month_history.columns if x not in ['card_id','month_lag']]
clf_stats={}
for col in cols:
    clf_stats[col]=stats
    
card_summary_month_history=card_summary_month_history.groupby(['card_id']).agg(clf_stats)
card_summary_month_history.columns=[x[0]+"_"+x[1]+'hist' for x in card_summary_month_history.columns]
card_summary_month_history=card_summary_month_history.reset_index()
card_summary_month_history.to_csv("../Fresh/card_summary_month_history.csv",index=False)

Wall time: 11min 51s


In [83]:
card_summary_month_history.head()

,card_id,authorized_sum_meanhist,authorized_sum_maxhist,authorized_sum_minhist,authorized_sum_<lambda>hist,city_id_nunique_meanhist,city_id_nunique_maxhist,city_id_nunique_minhist,city_id_nunique_<lambda>hist,installments_sum_meanhist,...,subsector_id_nunique_minhist,subsector_id_nunique_<lambda>hist,week_nunique_meanhist,week_nunique_maxhist,week_nunique_minhist,week_nunique_<lambda>hist,dayofyear_nunique_meanhist,dayofyear_nunique_maxhist,dayofyear_nunique_minhist,dayofyear_nunique_<lambda>hist
0,C_ID_00007093c1,8.769231,14,4,10,2.230769,3,1,2,14.769231,...,2,5,3.461538,5,1,4,6.230769,11,2,9
1,C_ID_0001238066,20.000000,35,2,33,6.333333,12,1,11,33.000000,...,1,12,4.000000,5,1,4,11.333333,16,2,14
2,C_ID_0001506ef0,4.769231,17,1,16,1.153846,2,1,1,0.076923,...,1,6,2.307692,4,1,3,3.384615,9,1,8
3,C_ID_0001793786,18.900000,35,2,33,4.000000,7,1,6,0.500000,...,2,11,3.500000,5,1,4,8.400000,13,1,12
4,C_ID_000183fdda,19.571429,28,0,28,3.000000,5,1,4,37.714286,...,3,11,4.285714,5,3,2,11.000000,16,3,13


##### Quarter Wise card purchases

In [88]:
bins=[-13,-10,-7,-4,0,3]
labels=['q5','q4','q3','q2','q1']

In [92]:
%%time
transactions3['quarter']=pd.cut(transactions3['month_lag'],bins=bins,labels=labels,include_lowest=True).astype(str)
quarter_purchase=transactions3.groupby(['card_id','quarter']).agg({'purchase_amount':'sum','purchase_date':'count'}).reset_index()
quarter_purchase_sum=pd.pivot_table(quarter_purchase,columns='quarter',values='purchase_amount',index='card_id',aggfunc='sum').reset_index()
quarter_purchase_sum.to_csv("../Fresh/quarter_purchase_sum.csv",index=False)
quarter_purchase_count=pd.pivot_table(quarter_purchase,columns='quarter',values='purchase_date',index='card_id',aggfunc='sum').reset_index()
quarter_purchase_count.to_csv("../Fresh/quarter_purchase_count.csv",index=False)

Wall time: 37.3 s


### Attaching with merchants data

In [93]:
merchant=pd.read_csv("../data/merchants.csv")

In [116]:
%%time
transactions4=pd.merge(transactions3[['card_id','merchant_id','merchant_category_id','purchase_amount']],
                       merchant[['merchant_id','merchant_group_id','numerical_1','numerical_2','most_recent_sales_range','most_recent_purchases_range']],
                      how='left',on='merchant_id')

Wall time: 40.5 s


In [114]:
clf={
     'numerical_1':['sum', 'median','max','min',lambda x: max(x)-min(x)],
     'numerical_2':['sum', 'median','max','min',lambda x: max(x)-min(x)],
     'merchant_group_id':'nunique'}

In [115]:
%%time
merchant_joint_analysis=transactions4.groupby('card_id').agg(clf)
merchant_joint_analysis.columns=[x[0]+"_"+x[1] for x in merchant_joint_analysis.columns]
merchant_joint_analysis=merchant_joint_analysis.reset_index()
merchant_joint_analysis.to_csv("../Fresh/merchant_joint_analysis.csv",index=False)

Wall time: 1min 36s


In [123]:
%%time
# transactions4['most_recent_sales_range']=transactions4['most_recent_sales_range'].fillna(0)
most_recent_sales_range=transactions4.groupby(['card_id','most_recent_sales_range']).\
    agg({'merchant_category_id':'nunique','merchant_id':'nunique','purchase_amount':'sum','numerical_1':'count'}).reset_index()

for i in most_recent_sales_range['most_recent_sales_range'].unique():
    most_recent_sales_range_2=most_recent_sales_range[most_recent_sales_range['most_recent_sales_range']==i]
    most_recent_sales_range_2.drop('most_recent_sales_range',inplace=True,axis=1)
    most_recent_sales_range_2.columns=[str(x)+str(i) for x in most_recent_sales_range_2.columns]
    most_recent_sales_range_2=most_recent_sales_range_2.rename(columns={'card_id'+str(i):'card_id'})
    most_recent_sales_range_2.to_csv("../Fresh/most_recent_sales_range_all"+str(i)+".csv", index=False)

C:\Users\sai.allaka\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Wall time: 2min


In [124]:
%%time
transactions4['most_recent_purchases_range']=transactions4['most_recent_purchases_range'].fillna(0)
most_recent_purchases_range=transactions4.groupby(['card_id','most_recent_purchases_range']).\
    agg({'merchant_category_id':'nunique','merchant_id':'nunique','purchase_amount':'sum','numerical_1':'count'}).reset_index()

for i in most_recent_purchases_range['most_recent_purchases_range'].unique():
    most_recent_purchases_range_2=most_recent_purchases_range[most_recent_purchases_range['most_recent_purchases_range']==i]
    most_recent_purchases_range_2.drop('most_recent_purchases_range',inplace=True,axis=1)
    most_recent_purchases_range_2.columns=[str(x)+str(i) for x in most_recent_purchases_range_2.columns]
    most_recent_purchases_range_2=most_recent_purchases_range_2.rename(columns={'card_id'+str(i):'card_id'})
    most_recent_purchases_range_2.to_csv("../Fresh/most_recent_purchases_range_all"+str(i)+".csv", index=False)

C:\Users\sai.allaka\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Wall time: 2min 33s


In [118]:
del[transactions1,transactions2,transactions3]
import gc
gc.collect()

332

### Data Understanding

#### Month_Lag

In [8]:
transactions1['month_lag'].unique()

array([ -8,  -7,  -6,  -5, -11,   0,  -3,  -9,  -4,  -1, -13, -10, -12,  -2], dtype=int64)

In [9]:
transactions2['month_lag'].unique()

array([1, 2], dtype=int64)

In [61]:
transactions3['category_2'].unique()

array([ 1.,  0.,  3.,  5.,  2.,  4.])

In [63]:
category2['category_2'].unique()

array([ 0.,  1.,  3.,  5.,  2.,  4.])

In [64]:
for i in category2['category_2'].unique():
    category2_2=category2[category2['category_2']==i]
    category2_2.drop('category_2',inplace=True,axis=1)
    category2_2.columns=[str(x)+str(i) for x in category2_2.columns]
    category2_2=category2_2.rename(columns={'card_id'+str(i):'card_id'})
    category2_2.to_csv("../Fresh/category2_all"+str(i)+".csv", index=False)

C:\Users\sai.allaka\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [87]:
transactions3['month_lag'].unique()

array([ -8,  -7,  -6,  -5, -11,   0,  -3,  -9,  -4,  -1, -13, -10, -12,
        -2,   1,   2], dtype=int64)

In [101]:
transactions3['category_3'].unique()

array(['A', 'B', 'C', nan], dtype=object)

In [104]:
merchant['merchant_group_id'].nunique()

109391

In [113]:
merchant.dtypes

merchant_id                     object
merchant_group_id                int64
merchant_category_id             int64
subsector_id                     int64
numerical_1                    float64
numerical_2                    float64
category_1                      object
most_recent_sales_range         object
most_recent_purchases_range     object
avg_sales_lag3                 float64
avg_purchases_lag3             float64
active_months_lag3               int64
avg_sales_lag6                 float64
avg_purchases_lag6             float64
active_months_lag6               int64
avg_sales_lag12                float64
avg_purchases_lag12            float64
active_months_lag12              int64
category_4                      object
city_id                          int64
state_id                         int64
category_2                     float64
dtype: object